# NBA Player Performance Dynamics: Dynamical Systems Analysis

This notebook applies dynamical systems theory to NBA player performance data, extracting deeper insights about player consistency, volatility, and predictability that are invisible to conventional statistics.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime
from statsmodels.tsa.api import VAR
from scipy import stats

# Add the project root to the path so we can import our modules
sys.path.append('..')

# Import our modules
from src.dynamics import (
    calculate_var_model,
    calculate_stability_metrics,
    plot_stability_distribution,
    calculate_performance_prediction,
    decompose_performance_factors,
    plot_performance_decomposition
)
from src.visualization import create_player_stability_quadrant
from src.utils import setup_plotting_style

# Set up plotting style
setup_plotting_style()

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## Dynamical Systems Theory Primer

Dynamical systems theory provides a mathematical framework for analyzing how systems evolve over time. In the context of NBA player performance, we can model a player's game-to-game performance as a dynamical system, where each game's statistics depend on previous games.

### Key Concepts

1. **State Space**: The set of all possible states of a system. For player performance, this includes metrics like points, rebounds, assists, and plus/minus.

2. **System Dynamics**: The rules governing how the system evolves from one state to another. We'll model this using Vector Autoregression (VAR).

3. **Stability**: How consistent a system's behavior is over time. Stable systems return to equilibrium after perturbations, while unstable systems amplify small changes.

4. **Lyapunov Exponents**: Measure the rate at which nearby trajectories in state space diverge. Positive exponents indicate chaos (high unpredictability), while negative exponents indicate stability.

5. **Entropy**: Quantifies the unpredictability or randomness in a system. Higher entropy indicates more unpredictable performance.

### Mathematical Foundations

For a discrete-time dynamical system, we can represent the state at time $t+1$ as a function of the state at time $t$:

$$X_{t+1} = f(X_t)$$

For player performance, we'll use a Vector Autoregression (VAR) model of order 1, which can be written as:

$$X_t = A X_{t-1} + \epsilon_t$$

where:
- $X_t$ is a vector of performance metrics at time $t$
- $A$ is a coefficient matrix that captures the system dynamics
- $\epsilon_t$ is a vector of error terms

The eigenvalues of matrix $A$ determine the stability of the system:
- If all eigenvalues have magnitude < 1, the system is stable
- If any eigenvalue has magnitude > 1, the system is unstable

The Lyapunov exponents are calculated as the natural logarithm of the absolute values of these eigenvalues:

$$\lambda_i = \ln|\mu_i|$$

where $\mu_i$ are the eigenvalues of $A$.

### Dynamical Systems in Basketball Context

In basketball, we can think of a player's performance as a dynamical system in several ways:

1. **Performance Momentum**: A player's current performance is influenced by their recent performances, creating a feedback loop.

2. **Stability vs. Volatility**: Some players have stable performance patterns that quickly return to their baseline after deviations, while others have volatile patterns that can swing dramatically from game to game.

3. **Predictability**: The entropy of a player's performance pattern indicates how predictable they are, which has strategic implications for both their team and opponents.

4. **System Interactions**: Players don't perform in isolation; their performance is part of a larger system that includes teammates, opponents, and game context.

By applying dynamical systems theory to player performance, we can extract insights that go beyond simple averages and variances, revealing the underlying patterns and dynamics that drive performance outcomes.

## Load Processed Data

Let's load the processed data from the previous notebook.

In [ ]:
# Load the processed data
try:
    player_temporal_df = pd.read_csv('../data/processed/player_temporal.csv')
    
    # Convert date strings to datetime objects
    player_temporal_df['GAME_DATE'] = pd.to_datetime(player_temporal_df['GAME_DATE'])
    
    print(f"Loaded player temporal data with {len(player_temporal_df)} records")
except FileNotFoundError:
    print("Processed data not found. Please run the 01_data_exploration.ipynb notebook first.")

In [ ]:
# Examine the structure of the temporal data
player_temporal_df.head()

In [ ]:
# Check the temporal features
temporal_features = ['PTS_MA5', 'PTS_Trend', 'PTS_Volatility', 'PTS_Change', 'PLUS_MINUS_Change', 'PTS_Momentum', 'Performance_Momentum']
player_temporal_df[temporal_features].describe()

## Player Performance System Modeling

Now let's model player performance as a dynamical system using Vector Autoregression (VAR).

### State Space Definition

For our dynamical system model, we'll define the state space using two key performance metrics:
1. **Points (PTS)**: Represents offensive production
2. **Plus/Minus (PLUS_MINUS)**: Represents overall impact on the game

These two metrics capture different aspects of player performance and will allow us to model how a player's scoring and overall impact evolve from game to game.

In [ ]:
# Select a sample player for detailed analysis
player_game_counts = player_temporal_df['Player_ID'].value_counts()
sample_player_id = player_game_counts.index[0]  # Player with most games
sample_player_name = player_temporal_df[player_temporal_df['Player_ID'] == sample_player_id]['PlayerName'].iloc[0]

# Get player's game data
player_data = player_temporal_df[player_temporal_df['Player_ID'] == sample_player_id].copy()
player_data = player_data.sort_values('GAME_DATE')

print(f"Selected {sample_player_name} for detailed analysis with {len(player_data)} games")

In [ ]:
# Visualize the state space (PTS vs PLUS_MINUS)
plt.figure(figsize=(10, 8))
plt.scatter(player_data['PTS'], player_data['PLUS_MINUS'], alpha=0.7)

# Add arrows to show transitions between consecutive games
for i in range(len(player_data) - 1):
    plt.arrow(player_data['PTS'].iloc[i], player_data['PLUS_MINUS'].iloc[i],
              player_data['PTS'].iloc[i+1] - player_data['PTS'].iloc[i],
              player_data['PLUS_MINUS'].iloc[i+1] - player_data['PLUS_MINUS'].iloc[i],
              head_width=0.5, head_length=0.5, fc='blue', ec='blue', alpha=0.3)

plt.xlabel('Points', fontsize=12)
plt.ylabel('Plus/Minus', fontsize=12)
plt.title(f'{sample_player_name}: Performance State Space', fontsize=14)
plt.grid(True, alpha=0.3)

# Add average point
avg_pts = player_data['PTS'].mean()
avg_pm = player_data['PLUS_MINUS'].mean()
plt.scatter(avg_pts, avg_pm, color='red', s=100, marker='*', label='Average')

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the time series of points and plus/minus
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Points time series
ax1.plot(player_data['GAME_DATE'], player_data['PTS'], marker='o', linestyle='-', label='Points')
ax1.plot(player_data['GAME_DATE'], player_data['PTS_MA5'], linestyle='-', color='red', label='5-Game Moving Avg')
ax1.set_ylabel('Points', fontsize=12)
ax1.set_title(f"{sample_player_name}: Game-by-Game Performance", fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plus/Minus time series
ax2.plot(player_data['GAME_DATE'], player_data['PLUS_MINUS'], marker='o', linestyle='-', color='green', label='Plus/Minus')
ax2.plot(player_data['GAME_DATE'], player_data['Performance_Momentum'], linestyle='-', color='purple', label='Performance Momentum')
ax2.set_xlabel('Game Date', fontsize=12)
ax2.set_ylabel('Plus/Minus', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='gray', linestyle='--')
ax2.legend()

plt.tight_layout()
plt.show()

### VAR Model Implementation

Now let's implement a Vector Autoregression (VAR) model to capture the dynamics of player performance.

In [ ]:
# Implement VAR model for the sample player
var_results, eigenvalues, lyapunov_exponents = calculate_var_model(player_data)

if var_results is not None:
    print(f"VAR Model Results for {sample_player_name}:")
    print(f"Coefficient Matrix:\n{var_results.coefs[0]}")
    print(f"\nEigenvalues: {eigenvalues}")
    print(f"Lyapunov Exponents: {lyapunov_exponents}")
    
    # Interpret stability
    max_eigenvalue = np.max(np.abs(eigenvalues))
    max_lyapunov = np.max(lyapunov_exponents)
    
    print(f"\nMax Eigenvalue Magnitude: {max_eigenvalue:.4f}")
    print(f"Max Lyapunov Exponent: {max_lyapunov:.4f}")
    
    if max_eigenvalue < 1:
        print("System is STABLE (eigenvalues < 1)")
    else:
        print("System is UNSTABLE (eigenvalues > 1)")
        
    if max_lyapunov < 0:
        print("System is CONVERGENT (negative Lyapunov exponents)")
    else:
        print("System shows CHAOTIC behavior (positive Lyapunov exponents)")
else:
    print("Could not calculate VAR model for the sample player.")

In [ ]:
# Visualize the VAR model predictions
if var_results is not None:
    # Extract features for VAR model
    var_data = player_data[['PTS', 'PLUS_MINUS']].copy()
    
    # Make predictions
    lag = 1
    predictions = var_results.fittedvalues
    
    # Plot actual vs. predicted values
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Points
    ax1.plot(player_data['GAME_DATE'][lag:], var_data['PTS'][lag:], marker='o', linestyle='-', label='Actual')
    ax1.plot(player_data['GAME_DATE'][lag:], predictions[:, 0], marker='x', linestyle='--', color='red', label='Predicted')
    ax1.set_ylabel('Points', fontsize=12)
    ax1.set_title(f"{sample_player_name}: VAR Model Predictions", fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Plus/Minus
    ax2.plot(player_data['GAME_DATE'][lag:], var_data['PLUS_MINUS'][lag:], marker='o', linestyle='-', label='Actual')
    ax2.plot(player_data['GAME_DATE'][lag:], predictions[:, 1], marker='x', linestyle='--', color='red', label='Predicted')
    ax2.set_xlabel('Game Date', fontsize=12)
    ax2.set_ylabel('Plus/Minus', fontsize=12)
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Calculate prediction errors
if var_results is not None:
    # Calculate errors
    pts_errors = var_data['PTS'][lag:].values - predictions[:, 0]
    pm_errors = var_data['PLUS_MINUS'][lag:].values - predictions[:, 1]
    
    # Calculate error metrics
    pts_rmse = np.sqrt(np.mean(pts_errors**2))
    pts_mae = np.mean(np.abs(pts_errors))
    pm_rmse = np.sqrt(np.mean(pm_errors**2))
    pm_mae = np.mean(np.abs(pm_errors))
    
    print(f"Points Prediction Errors - RMSE: {pts_rmse:.2f}, MAE: {pts_mae:.2f}")
    print(f"Plus/Minus Prediction Errors - RMSE: {pm_rmse:.2f}, MAE: {pm_mae:.2f}")
    
    # Plot error distributions
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Points errors
    sns.histplot(pts_errors, kde=True, ax=ax1)
    ax1.set_xlabel('Points Prediction Error', fontsize=12)
    ax1.set_title('Distribution of Points Prediction Errors', fontsize=14)
    ax1.axvline(x=0, color='red', linestyle='--')
    
    # Plus/Minus errors
    sns.histplot(pm_errors, kde=True, ax=ax2)
    ax2.set_xlabel('Plus/Minus Prediction Error', fontsize=12)
    ax2.set_title('Distribution of Plus/Minus Prediction Errors', fontsize=14)
    ax2.axvline(x=0, color='red', linestyle='--')
    
    plt.tight_layout()
    plt.show()

### Eigenvalue Calculation and Interpretation

The eigenvalues of the coefficient matrix in our VAR model provide crucial information about the stability of a player's performance. Let's examine these eigenvalues for multiple players to understand the range of stability profiles in the NBA.

In [ ]:
# Calculate eigenvalues for multiple players
player_eigenvalues = []

# Select players with enough games for meaningful analysis
min_games = 20
qualified_players = player_temporal_df['Player_ID'].value_counts()
qualified_players = qualified_players[qualified_players >= min_games].index

# Sample a subset of players for demonstration
import random
sample_players = random.sample(list(qualified_players), min(10, len(qualified_players)))

for player_id in sample_players:
    player_data = player_temporal_df[player_temporal_df['Player_ID'] == player_id].copy()
    player_name = player_data['PlayerName'].iloc[0]
    player_data = player_data.sort_values('GAME_DATE')
    
    var_results, eigenvalues, lyapunov_exponents = calculate_var_model(player_data)
    
    if var_results is not None and eigenvalues is not None:
        max_eigenvalue = np.max(np.abs(eigenvalues))
        max_lyapunov = np.max(lyapunov_exponents)
        
        player_eigenvalues.append({
            'player_id': player_id,
            'player_name': player_name,
            'games_played': len(player_data),
            'eigenvalues': eigenvalues,
            'max_eigenvalue': max_eigenvalue,
            'lyapunov_exponents': lyapunov_exponents,
            'max_lyapunov': max_lyapunov
        })

# Convert to dataframe
eigenvalue_df = pd.DataFrame([
    {
        'player_id': p['player_id'],
        'player_name': p['player_name'],
        'games_played': p['games_played'],
        'max_eigenvalue': p['max_eigenvalue'],
        'max_lyapunov': p['max_lyapunov']
    } for p in player_eigenvalues
])

# Sort by max eigenvalue
eigenvalue_df = eigenvalue_df.sort_values('max_eigenvalue')

# Display results
eigenvalue_df

In [ ]:
# Visualize the distribution of eigenvalues
plt.figure(figsize=(10, 6))
plt.hist(eigenvalue_df['max_eigenvalue'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(x=1, color='red', linestyle='--', label='Stability Threshold')
plt.xlabel('Maximum Eigenvalue Magnitude', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Stability (Eigenvalues)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot eigenvalues in the complex plane
plt.figure(figsize=(10, 10))

# Draw unit circle
circle = plt.Circle((0, 0), 1, fill=False, color='red', linestyle='--', label='Unit Circle')
plt.gca().add_patch(circle)

# Plot eigenvalues for each player
for player in player_eigenvalues:
    for eigenvalue in player['eigenvalues']:
        plt.scatter(eigenvalue.real, eigenvalue.imag, alpha=0.7)
    
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.xlabel('Real Part', fontsize=12)
plt.ylabel('Imaginary Part', fontsize=12)
plt.title('Eigenvalues in the Complex Plane', fontsize=14)
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.legend()
plt.tight_layout()
plt.show()

### Interpreting Eigenvalues and Lyapunov Exponents

The eigenvalues and Lyapunov exponents provide important insights about player performance dynamics:

1. **Eigenvalues < 1 (Stable Systems)**:
   - Players with all eigenvalues less than 1 in magnitude have stable performance systems
   - Their performance tends to return to baseline after deviations
   - These players are more predictable and reliable

2. **Eigenvalues > 1 (Unstable Systems)**:
   - Players with any eigenvalue greater than 1 in magnitude have unstable performance systems
   - Their performance can amplify small deviations, leading to streaky patterns
   - These players are less predictable and more volatile

3. **Negative Lyapunov Exponents (Convergent Systems)**:
   - Players with negative Lyapunov exponents have performance trajectories that converge
   - Similar initial conditions lead to similar outcomes
   - These players have more consistent performance patterns

4. **Positive Lyapunov Exponents (Chaotic Systems)**:
   - Players with positive Lyapunov exponents have performance trajectories that diverge
   - Similar initial conditions can lead to very different outcomes
   - These players have more chaotic and unpredictable performance patterns

The combination of eigenvalues and Lyapunov exponents allows us to classify players into different stability profiles, providing a more nuanced understanding of their performance dynamics than traditional statistics.

## Stability Metrics Extraction

Now let's calculate stability metrics for all players using our dynamics module.

In [ ]:
# Calculate stability metrics for all players
stability_df = calculate_stability_metrics(player_temporal_df)

print(f"Calculated stability metrics for {len(stability_df)} players")
stability_df.head()

In [ ]:
# Examine the distribution of stability metrics
plt.figure(figsize=(12, 8))
plt.hist(stability_df['system_stability'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')

# Add vertical lines for stability categories
plt.axvline(x=0.7, color='green', linestyle='--', label='Highly Stable')
plt.axvline(x=0.9, color='blue', linestyle='--', label='Stable')
plt.axvline(x=1.1, color='orange', linestyle='--', label='Moderately Stable')
plt.axvline(x=1.3, color='red', linestyle='--', label='Volatile')

plt.xlabel('System Stability Score (lower = more stable)', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Performance Stability', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the percentage of players in each stability category
stability_counts = stability_df['stability_type'].value_counts()
stability_percentages = stability_counts / len(stability_df) * 100

# Display results
stability_percentages

In [ ]:
# Visualize the distribution of stability types
plt.figure(figsize=(10, 6))
stability_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Stability Type', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Stability Types', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### System Stability Calculation

Our system stability metric combines several aspects of player performance dynamics:

1. **Maximum Eigenvalue**: Captures the overall stability of the dynamical system
2. **Maximum Lyapunov Exponent**: Measures the rate of divergence of nearby trajectories
3. **Points Volatility**: Quantifies the game-to-game variability in scoring
4. **Performance Entropy**: Measures the unpredictability of performance

The formula for system stability is:

```python
system_stability = 0.3 * max_eigenvalue + 0.3 * max_lyapunov + 0.2 * pts_volatility + 0.2 * (performance_entropy / 3)
```

Lower values indicate more stable performance, while higher values indicate more volatile or chaotic performance.

In [ ]:
# Examine the relationship between stability components
stability_components = ['max_eigenvalue', 'max_lyapunov', 'pts_volatility', 'performance_entropy', 'system_stability']

# Calculate correlation matrix
stability_corr = stability_df[stability_components].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(stability_corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Stability Metrics', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the relationship between stability components
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Eigenvalue vs. Lyapunov
axes[0, 0].scatter(stability_df['max_eigenvalue'], stability_df['max_lyapunov'], alpha=0.7)
axes[0, 0].set_xlabel('Maximum Eigenvalue', fontsize=12)
axes[0, 0].set_ylabel('Maximum Lyapunov Exponent', fontsize=12)
axes[0, 0].set_title('Eigenvalue vs. Lyapunov Exponent', fontsize=14)
axes[0, 0].grid(True, alpha=0.3)

# Volatility vs. Entropy
axes[0, 1].scatter(stability_df['pts_volatility'], stability_df['performance_entropy'], alpha=0.7)
axes[0, 1].set_xlabel('Points Volatility', fontsize=12)
axes[0, 1].set_ylabel('Performance Entropy', fontsize=12)
axes[0, 1].set_title('Volatility vs. Entropy', fontsize=14)
axes[0, 1].grid(True, alpha=0.3)

# Eigenvalue vs. System Stability
axes[1, 0].scatter(stability_df['max_eigenvalue'], stability_df['system_stability'], alpha=0.7)
axes[1, 0].set_xlabel('Maximum Eigenvalue', fontsize=12)
axes[1, 0].set_ylabel('System Stability', fontsize=12)
axes[1, 0].set_title('Eigenvalue vs. System Stability', fontsize=14)
axes[1, 0].grid(True, alpha=0.3)

# Entropy vs. System Stability
axes[1, 1].scatter(stability_df['performance_entropy'], stability_df['system_stability'], alpha=0.7)
axes[1, 1].set_xlabel('Performance Entropy', fontsize=12)
axes[1, 1].set_ylabel('System Stability', fontsize=12)
axes[1, 1].set_title('Entropy vs. System Stability', fontsize=14)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Lyapunov Exponent Computation

Lyapunov exponents measure the rate at which nearby trajectories in state space diverge. In the context of player performance, they quantify how quickly small differences in initial conditions lead to large differences in outcomes.

- **Negative Lyapunov exponents** indicate that trajectories converge, suggesting stable, predictable performance.
- **Positive Lyapunov exponents** indicate that trajectories diverge, suggesting chaotic, unpredictable performance.

We calculate Lyapunov exponents as the natural logarithm of the absolute values of the eigenvalues of the coefficient matrix in our VAR model.

In [ ]:
# Examine the distribution of Lyapunov exponents
plt.figure(figsize=(10, 6))
plt.hist(stability_df['max_lyapunov'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(x=0, color='red', linestyle='--', label='Chaos Threshold')
plt.xlabel('Maximum Lyapunov Exponent', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Lyapunov Exponents', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the percentage of players with positive vs. negative Lyapunov exponents
positive_lyapunov = (stability_df['max_lyapunov'] > 0).sum()
negative_lyapunov = (stability_df['max_lyapunov'] <= 0).sum()

print(f"Players with positive Lyapunov exponents (chaotic): {positive_lyapunov} ({positive_lyapunov / len(stability_df) * 100:.1f}%)")
print(f"Players with negative Lyapunov exponents (stable): {negative_lyapunov} ({negative_lyapunov / len(stability_df) * 100:.1f}%)")

### Performance Entropy Calculation

Performance entropy quantifies the unpredictability or randomness in a player's performance. Higher entropy indicates more unpredictable performance, while lower entropy indicates more predictable performance.

We calculate performance entropy using the formula:

$$H = -\sum_{i} p_i \log_2(p_i)$$

where $p_i$ is the probability of the player's performance falling into bin $i$ of a histogram of normalized performance values.

In [ ]:
# Examine the distribution of performance entropy
plt.figure(figsize=(10, 6))
plt.hist(stability_df['performance_entropy'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Performance Entropy', fontsize=12)
plt.ylabel('Number of Players', fontsize=12)
plt.title('Distribution of Player Performance Entropy', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot entropy vs. points volatility
plt.figure(figsize=(10, 8))
plt.scatter(stability_df['performance_entropy'], stability_df['pts_volatility'], 
            alpha=0.7, c=stability_df['system_stability'], cmap='coolwarm')

# Add labels for notable players
for i, row in stability_df.nlargest(5, 'performance_entropy').iterrows():
    plt.annotate(row['player_name'], 
                 (row['performance_entropy'], row['pts_volatility']),
                 fontsize=9)

plt.xlabel('Performance Entropy', fontsize=12)
plt.ylabel('Points Volatility', fontsize=12)
plt.title('Player Entropy vs. Volatility', fontsize=14)
plt.colorbar(label='System Stability')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Prepare Data for Player Classification

Now let's prepare the data for player classification by combining stability metrics with traditional performance statistics.

In [ ]:
# Calculate average points and plus/minus for each player
player_stats = player_temporal_df.groupby('Player_ID').agg({
    'PlayerName': 'first',
    'PTS': 'mean',
    'PLUS_MINUS': 'mean'
}).reset_index()

# Merge with stability metrics
player_analysis = pd.merge(
    stability_df,
    player_stats,
    on=['Player_ID', 'player_name'],
    how='inner'
)

# Rename columns for clarity
player_analysis = player_analysis.rename(columns={
    'PTS': 'avg_pts',
    'PLUS_MINUS': 'avg_plus_minus'
})

# Display the merged data
player_analysis.head()

In [ ]:
# Plot stability vs. points
plt.figure(figsize=(12, 8))
plt.scatter(player_analysis['system_stability'], player_analysis['avg_pts'], 
            alpha=0.7, c=player_analysis['avg_plus_minus'], cmap='RdYlGn')

# Add labels for notable players
for i, row in player_analysis.nlargest(5, 'avg_pts').iterrows():
    plt.annotate(row['player_name'], 
                 (row['system_stability'], row['avg_pts']),
                 fontsize=9)
    
for i, row in player_analysis.nsmallest(5, 'system_stability').iterrows():
    plt.annotate(row['player_name'], 
                 (row['system_stability'], row['avg_pts']),
                 fontsize=9)

plt.xlabel('System Stability (lower = more stable)', fontsize=12)
plt.ylabel('Points Per Game', fontsize=12)
plt.title('Player Stability vs. Scoring', fontsize=14)
plt.colorbar(label='Plus/Minus')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Save Results for Subsequent Analysis

Let's save our player analysis results for use in subsequent notebooks.

In [ ]:
# Save player analysis results
player_analysis.to_csv('../data/processed/player_dynamics.csv', index=False)
print(f"Saved player dynamics analysis to ../data/processed/player_dynamics.csv")

## Conclusion

In this notebook, we've applied dynamical systems theory to NBA player performance data, extracting deeper insights about player consistency, volatility, and predictability. We've developed a comprehensive framework for analyzing player performance dynamics using Vector Autoregression models, eigenvalues, Lyapunov exponents, and entropy calculations.

Key accomplishments:
1. Modeled player performance as a dynamical system using Vector Autoregression
2. Calculated stability metrics including eigenvalues, Lyapunov exponents, and performance entropy
3. Developed a system stability score that combines multiple aspects of performance dynamics
4. Prepared data for player classification and basketball interpretation

In the next notebook (02b_player_stability_analysis.ipynb), we'll build on these results to classify players into distinct stability profiles, examine case studies of different player types, and translate our mathematical findings into practical basketball insights.